In [1]:
from pyspark.sql.functions import lit

# Load in one of the tables
df1 = spark.sql("select * from default.video_games_5")
df1 = df1.withColumn('category', lit("video_games"))

df2 = spark.sql("select * from default.home_and_kitchen_5_small")
df2 = df2.withColumn('category', lit("home_and_kitchen"))

df3 = spark.sql("select * from default.books_5_small")
df3 = df3.withColumn('category', lit("books"))

df = df1.union(df2).union(df3)

# Take a sample (useful for code development purposes)
df = df.sample(False, 0.15, seed=0)

df = df.cache()

print((df.count(), len(df.columns)))

(522776, 13)

In [2]:
df.printSchema()

root
-- reviewID: long (nullable = true)
-- overall: double (nullable = true)
-- vote: integer (nullable = true)
-- verified: boolean (nullable = true)
-- reviewTime: string (nullable = true)
-- reviewerID: string (nullable = true)
-- asin: string (nullable = true)
-- reviewerName: string (nullable = true)
-- reviewText: string (nullable = true)
-- summary: string (nullable = true)
-- unixReviewTime: integer (nullable = true)
-- label: integer (nullable = true)
-- category: string (nullable = false)

In [3]:
# Let's look at some quick summary statistics
df.describe().show()

+-------+-------------------+------------------+------------------+----------+--------------------+--------------------+------------+--------------------+--------------------+--------------------+-------------------+
summary| reviewID| overall| vote|reviewTime| reviewerID| asin|reviewerName| reviewText| summary| unixReviewTime| label|
+-------+-------------------+------------------+------------------+----------+--------------------+--------------------+------------+--------------------+--------------------+--------------------+-------------------+
 count| 522776| 522776| 522776| 522776| 522776| 522776| 522750| 522776| 522719| 522776| 522776|
 mean|6.359225185982467E8| 4.321015119286272|1.9612204844904892| null| null|4.4575601534272656E7| Infinity| 67.8| 26942.585263157893| 1.39232038365648E9|0.18037744655454727|
 stddev|2.247665725992215E9|1.1242426352591086|18.300195627572595| null| null|2.7765518536187214E8| NaN| 298.7386817939719| 162287.9780946834|1.0328092483814336E8|0.38450189353273456|
 min| 0| 1.0| 0|01 1, 1998|A0001528BGUBOEVR6T5U| 0000913154| | I Scream, You S...| (^'.'^) <('.'<) ...| 843004800| 0|
 max| 8590192388| 5.0| 5777|12 9, 2017| AZZZYAYJQSDOJ| B01HIZF7XE| ~~PaulB|~~~~ I honestly ...|~~~ One of my FAV...| 1537660800| 1|
+-------+-------------------+------------------+------------------+----------+--------------------+--------------------+------------+--------------------+--------------------+--------------------+-------------------+

In [4]:
display(df.groupBy("category").count().orderBy("category"))

category,count
books,149471
home_and_kitchen,300223
video_games,73082


In [5]:
from pyspark.sql.functions import col
display(df.groupBy("overall").count().orderBy("overall"))

overall,count
1.0,26813
2.0,22995
3.0,43537
4.0,91646
5.0,337785


In [6]:
# The most common product IDs
display(df.groupBy("asin").count().orderBy(col("count").desc()).head(50))

asin,count
0007420412,2919
000711835X,2847
0007548672,2345
0007350899,1172
0007444117,1164
B000YGEVMI,1029
0007378033,930
B0015TMHSI,892
006017322X,780
0007350783,733


In [7]:
display(df.groupBy("label").count().orderBy("label")))

label,count
0,428479
1,94297


In [8]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover
from pyspark.sql import functions as f

# We'll tokenize the text using a simple RegexTokenizer
tokenizer = RegexTokenizer(inputCol="reviewText", outputCol="words", pattern="\\W")

# Remove standard Stopwords
stopwordsRemover = StopWordsRemover(inputCol="words", outputCol="filtered")

pipeline = Pipeline(stages=[tokenizer, stopwordsRemover])

pipelineFit = pipeline.fit(df)
df = pipelineFit.transform(df)

In [9]:
counts = df.select(f.explode('filtered').alias('col')).groupBy('col').count().sort(f.desc('count')).collect()
display(counts)

col,count
one,187045
book,184569
game,152816
great,146841
like,144142
good,118181
well,104851
read,99594
time,92205
get,89471
